# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../output_data/cities.csv")
vacation_df.dropna()
vacation_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
locations = vacation_df[["Lat","Lng"]]
humidity = vacation_df["Humidity"]

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False, max_intensity=10,point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#hotel_df = cities_df[ ( (cities_df["temp_min"]>72) & (cities_df["wind_speed"]<10) & (cities_df["cloudiness"]<10)]

temp_vacation = vacation_df.loc[(vacation_df['Max Temp']>=70)&(vacation_df['Max Temp']<=80)]
wind_file = temp_vacation.loc[(temp_vacation['Wind Speed']<=10)]
weather = wind_file.loc[(wind_file['Cloudiness']<=10)]
weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = weather.reset_index()
hotel_df['Hotel Name']= ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params ={
        "radius":5000,
        "types": "Hotel",
        "keyword": "Lodging",
        "key":g_key
}
for index,row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url,params=params).json()
    results = response['results']
    #print(json.dumps(results, indent=4, sort_keys=True))
    try:
        hotel_df.loc[index,"Hotel Name"] = results[0]['name']
    except:
        print(f"Hotel not found")
    continue
print(hotel_df)
    
    

Hotel not found
Hotel not found
    index  City_ID          City  Cloudiness Country        Date  Humidity  \
0       7        7         opuwo           0     NaN  1558378523        23   
1      70       70  ponta do sol           0      BR  1558378767        57   
2      87       87      ambilobe           2      MG  1558378770        92   
3      88       88      cidreira           0      BR  1558378770        66   
4     138      138     morondava           0      MG  1558378780        69   
5     139      139         vaini           0      IN  1558378780        88   
6     176      176         nador           0      MA  1558378499        40   
7     268      268        dingle           4      PH  1558378836        80   
8     297      297         mogok           0      MM  1558378844        51   
9     402      402       sambava           7      MG  1558378871        99   
10    444      444       birjand           0      IR  1558378880        26   
11    494      494         baylo

In [8]:
hotel_result = pd.DataFrame(hotel_df)
hotel_result

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge
1,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Camaleao Hostel
2,87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,Hotel National
3,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Pousada Lessa
4,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
5,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,woodpeckerjunglecamp
6,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,"Hotel Marchica Lagoon Resort, Nador Morocco"
7,268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73,
8,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,King Bridge
9,402,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50,Carrefour


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))